In [ ]:
#Uses python3

import numpy as np
import time
import pandas as pd
import sys

In [100]:
#====================== all functions ======================
def coord(site):
    """get coordinate i of vector"""
    x = site // L
    y = site - x*L
    return (x,y)

def get(i):
    """fixin' boundary"""
    if i<0: return i
    else: return i % L
    
def get_neigh():
    """get neighbour's arr"""
    s = np.arange(L**2).reshape(L,L)
    nei = []
    for site in range(L*L):
        i,j = coord(site)
        nei += [s[get(i-1),get(j)],s[get(i),get(j+1)],s[get(i+1),get(j)],s[get(i),get(j-1)]]
    return np.array(nei, dtype=np.int32).reshape(L*L,4)

#################################################################
def gen_state():
    """generate random start state with lenght L*L and [-1,1] components"""
    state_s = np.array([np.random.choice([-1,1]) for _ in range(L*L)], dtype=np.int32)#.reshape(L,L)
    state_t = np.array([np.random.choice([-1,1]) for _ in range(L*L)], dtype=np.int32)#.reshape(L,L)
    return state_s, state_t

def mc_choice(dE,T):
    """принимаем или не принимаем переворот спина?"""
    if dE < 0:
        return True
    elif np.random.uniform() <= np.exp(-dE/T):
        return True
    else:
        return False 
    
def calc_dE(site_s, site_t, neigh_s, neigh_t,l):
    """calculate dE"""
    e1,e2 = 0,0
    for i in range(4):
        e1 += (site_s*neigh_s[i] + site_t*neigh_t[i] + site_s*neigh_s[i]*site_t*neigh_t[i])
    if l==1:
        site_s *= -1
        for i in range(4):
            e2 += (site_s*neigh_s[i] + site_t*neigh_t[i] + site_s*neigh_s[i]*site_t*neigh_t[i])
    else:
        site_t *= -1
        for i in range(4):
            e2 += (site_s*neigh_s[i] + site_t*neigh_t[i] + site_s*neigh_s[i]*site_t*neigh_t[i])
    return e1-e2

def step(s,t,nei,T):
    """крутим 1 спин"""
    j,k = np.random.randint(-1,L-1), np.random.randint(-1,L-1) 
    if j==-1: j=L-1
    if k==-1: k=L-1
    site = j*L+k
    
    if np.random.choice([-1,1])==1:
        neigh_s = s[nei[site,:]]  
        neigh_t = t[nei[site,:]]  
        dE = calc_dE(s[site], t[site], neigh_s, neigh_t,1)
        if mc_choice(dE,T):
            s[site] *= -1
    else: 
        neigh_s = s[nei[site,:]]  
        neigh_t = t[nei[site,:]]  
        dE = calc_dE(s[site], t[site], neigh_s, neigh_t,-1)
        if mc_choice(dE,T):
            t[site] *= -1

def mc_step(s, t, nei, T):
    """perform 2*L*L steps MC to relax"""
    for _ in range(2*L*L):
        step(s,t,nei,T)
        
#################################################################
def expand(state):
    _a = np.concatenate((state[L-1].reshape(1,L), state), axis=0)   # add last line to TOP
    a = np.concatenate((_a, _a[:,0].reshape(L+1,1)), axis=1)    # add first line to RIGHT
    return a
def calc_e(s,t):
    s,t = s.reshape(L,L),t.reshape(L,L)
    s,t = expand(s), expand(t)
    e = 0
    for i in range(L):
        for j in range(L):
            e += s[i,j]*s[i+1,j] + t[i,j]*t[i+1,j] + s[i,j]*s[i+1,j]*t[i,j]*t[i+1,j] # right neighbour
            e += s[i,j]*s[i,j+1] + t[i,j]*t[i,j+1] + s[i,j]*s[i,j+1]*t[i,j]*t[i,j+1] # down neighbour
    return -e/(L*L)
def calc_m(s):
    m = 0
    for i in range(L):
        for j in range(L):
            m += s[i,j]
    return m/(L*L) 
#################################################################
def model_at(T,N_avg,N_mc=1,Relax=1000):
    """Моделируем АТ"""
    E, M_s, M_t = [], [], []
    edict = gen_e_dict()
    state_s, state_t = gen_state()
    
    #relax 10**3 times be4 AVG
    for __ in range(Relax):
            state_s, state_t = mc_step(state_s, state_t, edict, T)
    #AVG every N_mc steps
    for _ in range(N_avg):
        for __ in range(N_mc):
            state_s, state_t = mc_step(state_s, state_t, edict, T)
        E += [calc_e(state_s,state_t,L)]
        M_s += [calc_m(state_s,L)]
        M_t += [calc_m(state_t,L)]
    
    return E, M_s, M_t

In [102]:
global L
L = 100

np.random.seed(1)
s,t = gen_state(); nei = get_neigh()#gen_e_dict()

%timeit [mc_step(s,t,nei,1) for _ in range(10)]

13.6 s ± 498 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [101]:
global L
L = 100

np.random.seed(1)
s,t = gen_state(); nei = get_neigh()#gen_e_dict()
print(calc_e(s,t))

for _ in range(10):
    mc_step(s,t, nei, 1)
print(calc_e(s,t))

-0.0556
-4.226


In [ ]:
# py              -- 4.83 s ± 136 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
# py + neigh      -- 13.6 s ± 498 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

In [ ]:
-0.0556
-4.226